In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [2]:
training_data = datasets.FashionMNIST(  ##训练集
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

100.0%


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [3]:
test_data = datasets.FashionMNIST(   ##测试集
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [4]:
##将构建好的datasets作为dataloader的输入
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [6]:
##构建模型
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()    ##调用父类module
        self.flatten = nn.Flatten()              ##Flatten将张量从第一个维度开始都变为一个维度
        self.linear_relu_stack = nn.Sequential(  ##Sequential有序的容器，通过添加的module，有序的串联起来
            nn.Linear(28*28, 512),    #线性层
            nn.ReLU(),                #非线性激活函数
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    
    def forward(self, x):           ##前向运算
        x = self.flatten(x)         ##对输入的维度进行一个flatten操作
        logits = self.linear_relu_stack(x)   
        return logits
    

In [7]:
model = NeuralNetwork()  #实例化网络

In [8]:
#超参数
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [9]:
# 交叉熵损失函数
loss_fn = nn.CrossEntropyLoss()

In [10]:
##构建优化器
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# 在训练循环中，优化分三步进行：
1、调用以重置模型参数的梯度。默认情况下，渐变相加起来;为了防止重复计数，我们在每次迭代时显式地将它们归零。optimizer.zero_grad()

2、通过调用 反向传播预测损失。PyTorch 会沉积每个参数的损耗梯度。loss.backward()

3、一旦我们有了梯度，我们调用通过向后传递中收集的梯度来调整参数。optimizer.step()

In [11]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)        ##计算dataloader的大小
    for batch, (X, y) in enumerate(dataloader):  ##enumerate枚举函数
        # Compute prediction and loss
        pred = model(X)  ##logits
        loss = loss_fn(pred, y)  ##将logits和y放入loss_fn函数中得到loss标量

        # Backpropagation
        optimizer.zero_grad()   ##对每个参数梯度置零
        loss.backward()         ##loss对所有参数梯度的计算
        optimizer.step()        ##通过优化器对参数进行更新

        if batch % 100 == 0:  #每到100个就来查看一下loss的值
            loss, current = loss.item(), batch * len(X)  ##当前训练了多少个样本了
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [12]:
##进行周期训练
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.303466  [    0/60000]
loss: 2.288825  [ 6400/60000]
loss: 2.274044  [12800/60000]
loss: 2.263069  [19200/60000]
loss: 2.235757  [25600/60000]
loss: 2.228578  [32000/60000]
loss: 2.216583  [38400/60000]
loss: 2.189525  [44800/60000]
loss: 2.185098  [51200/60000]
loss: 2.144545  [57600/60000]
Test Error: 
 Accuracy: 54.6%, Avg loss: 2.145669 

Epoch 2
-------------------------------
loss: 2.158687  [    0/60000]
loss: 2.145877  [ 6400/60000]
loss: 2.089099  [12800/60000]
loss: 2.096604  [19200/60000]
loss: 2.044379  [25600/60000]
loss: 1.994059  [32000/60000]
loss: 2.001401  [38400/60000]
loss: 1.927449  [44800/60000]
loss: 1.931690  [51200/60000]
loss: 1.844731  [57600/60000]
Test Error: 
 Accuracy: 58.6%, Avg loss: 1.852291 

Epoch 3
-------------------------------
loss: 1.890171  [    0/60000]
loss: 1.857084  [ 6400/60000]
loss: 1.737202  [12800/60000]
loss: 1.768098  [19200/60000]
loss: 1.675373  [25600/60000]
loss: 1.623226  [32000/600